In [3]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
# pip install XlsxWriter

Defaulting to user installation because normal site-packages is not writeable
  Using cached xlsxwriter-3.2.5-py3-none-any.whl.metadata (2.7 kB)
Using cached xlsxwriter-3.2.5-py3-none-any.whl (172 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# starting with raw_ofac_list
df = pd.read_excel(r'C:\Users\wasia\Desktop\OFAC data listing\raw_ofac_list.xlsx')

# function to extract name and alias from ofac raw file

def extract_name_alias(text):
    if not isinstance(text, str) or not text.strip():
        return pd.Series({'name': '', 'alias': ''})
    
    all_brackets = re.findall(r'\(([^()]*)\)', text)
    if not all_brackets:
        return pd.Series({'name': '', 'alias': ''})
    
    first_bracket = all_brackets[0]
    
    # If first () contains Linked To:, return empty
    if 'Linked To:' in first_bracket:
        return pd.Series({'name': '', 'alias': ''})
    
    # If first () does NOT contain a.k.a. or f.k.a., and does NOT contain colon, return empty
    if ('a.k.a.' not in first_bracket and 'f.k.a.' not in first_bracket) and (':' not in first_bracket):
        return pd.Series({'name': '', 'alias': ''})
    
    name = text.split('(', 1)[0].strip()
    
    alias_list = []
    if 'a.k.a.' not in first_bracket and 'f.k.a.' not in first_bracket:
        alias_list.append(first_bracket.strip())
    
    aka_matches = re.findall(r'\(([^()]*(?:a\.k\.a\.|f\.k\.a\.).*?)\)', text)
    for match in aka_matches:
        parts = re.split(r'a\.k\.a\.|f\.k\.a\.', match)
        for part in parts[1:]:
            aliases = [p.strip() for p in part.split(';') if p.strip()]
            for alias in aliases:
                nested = re.findall(r'\(([^()]*)\)', alias)
                if nested:
                    alias_main = re.sub(r'\([^()]*\)', '', alias).strip()
                    alias_list.append(alias_main)
                    for n in nested:
                        alias_list.append(n.strip())
                else:
                    alias_list.append(alias)
    
    # Deduplicate aliases preserving order
    seen = set()
    final_aliases = []
    for a in alias_list:
        if a not in seen:
            seen.add(a)
            final_aliases.append(a)
    
    return pd.Series({'name': name, 'alias': '; '.join(final_aliases)})

# Usage example:

df[['name', 'alias']] = df['list'].apply(extract_name_alias)

# exporting to check file
df.to_excel('df_check.xlsx', index=False)




In [5]:
import pandas as pd

url = "https://data.trade.gov/downloadable_consolidated_screening_list/v1/consolidated.csv"

df_csv = pd.read_csv(url)

# Display first 5 rows
df_csv.head()

,_id,source,entity_number,type,programs,name,title,addresses,federal_register_notice,start_date,...,vessel_owner,remarks,source_list_url,alt_names,citizenships,dates_of_birth,nationalities,places_of_birth,source_information_url,ids
0,2446250b546e45b90dd23fc576a4f02f49339d19d9b2f0...,Nonproliferation Sanctions (ISN) - State Depar...,NaN,NaN,Chemical and Biological Weapons Act,SPC Supachoke,NaN,NaN,"Vol. 59, No. 44, 03/07/1994",1994-02-08,...,NaN,NaN,https://www.state.gov/key-topics-bureau-of-int...,Super Trade,NaN,NaN,NaN,NaN,https://www.state.gov/key-topics-bureau-of-int...,NaN
1,15268,Capta List (CAP) - Treasury Department,15268.0,Entity,561-Related,BANK OF KUNLUN CO LTD,NaN,"172 Xibin Rd, Ranghulu District, (Daqing, Heil...",NaN,NaN,...,NaN,United States financial institutions are prohi...,https://sanctionslist.ofac.treas.gov/Home/Cons...,KARAMAY URBAN CREDIT COOPERATIVES; KARAMAY CIT...,NaN,NaN,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"SWIFT/BIC, CKLBCNBJ"
2,30882,Non-SDN Chinese Military-Industrial Complex Co...,30882.0,Entity,CMIC-EO13959,China Telecom Corporation Limited,NaN,"31 Jinrong Street, Xicheng District, Beijing, ...",NaN,NaN,...,NaN,NaN,https://sanctionslist.ofac.treas.gov/Home/Cons...,"CHINA TELECOM; CHINA TELECOM CO., LTD; CHINA T...",NaN,NaN,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"Unified Social Credit Code (USCC), 91110000710..."
3,17013,Sectoral Sanctions Identifications List (SSI) ...,17013.0,Entity,UKRAINE-EO13662; RUSSIA-EO14024,VTB BANK PUBLIC JOINT STOCK COMPANY,NaN,"29, Bolshaya Morskaya str., St. Petersburg, 19...",NaN,NaN,...,NaN,"For more information on directives, please vis...",https://sanctionslist.ofac.treas.gov/Home/Cons...,VTB Bank PAO; VTB Bank PJSC; Bank VTB Publichn...,NaN,NaN,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"Registration ID, 1027739609391, RU; Government..."
4,17014,Non-SDN Menu-Based Sanctions List (NS-MBS List...,17014.0,Entity,UKRAINE-EO13662; RUSSIA-EO14024,Joint Stock Company Russian Agricultural Bank,NaN,"3, Gagarinsky Pereulok, Moscow, 119034, RU; 3 ...",NaN,NaN,...,NaN,"For more information on directives, please vis...",https://sanctionslist.ofac.treas.gov/Home/Cons...,Russian Agricultural Bank; Russian Agricultura...,NaN,NaN,NaN,NaN,https://ofac.treasury.gov/other-ofac-sanctions...,"Registration Number, 1027700342890, RU; Govern..."


In [6]:
df_csv.shape

(23300, 29)

In [7]:
# df_csv.to_csv('ofac.csv')

In [37]:
df_check = pd.read_excel('df_check.xlsx')
df_check

,list,source,name,alias
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731,"ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,..."
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司


In [38]:
# checking if df_check names found in df_csv
import pandas as pd

def match_names(df_check, df_csv):
    name_matched_list = []
    id_list = []
    
    for name in df_check["name"]:
        matches = df_csv[df_csv["name"] == name]
        
        if len(matches) == 1:
            name_matched_list.append(matches.iloc[0]["name"])
            id_list.append(matches.iloc[0]["_id"])
        
        elif len(matches) > 1:
            name_matched_list.append("multiple_matches_need_to_check")
            id_list.append(None)
        
        else:
            name_matched_list.append(None)
            id_list.append(None)
    
    df_check["name_matched"] = name_matched_list
    df_check["id"] = id_list
    return df_check
# applying
df_check = match_names(df_check, df_csv)
df_check



,list,source,name,alias,name_matched,id
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731,"ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,...","ALIZADEH HOSHYAR, Javad",54930
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,None,None
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,CONTROL AFZAR TABRIZ CO LTD,54926
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,None,None
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,MECATRON MACHINERY CO LTD,54943


In [39]:
# if all values not found recheck the df_check
df_check.to_excel('df_checkv2.xlsx', index=False)

In [47]:
df_checkv2 = pd.read_excel('df_checkv2.xlsx', dtype={'id': 'Int64'})
df_checkv2

,list,source,name,alias,name_matched,id
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731,"ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,...","ALIZADEH HOSHYAR, Javad",54930
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,NaN,<NA>
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,CONTROL AFZAR TABRIZ CO LTD,54926
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,NaN,<NA>
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,MECATRON MACHINERY CO LTD,54943


In [41]:
df_checkv2.shape

(6, 6)

In [48]:
# combining id and name column
df_checkv2["id-name"] = df_checkv2["id"].astype("string").fillna('') + "-" + df_checkv2["name"].fillna('').astype(str)
df_checkv2

,list,source,name,alias,name_matched,id,id-name
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731,"ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,...","ALIZADEH HOSHYAR, Javad",54930,"54930-ALIZADEH HOSHYAR, Javad"
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,54945,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,NaN,<NA>,-
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,CONTROL AFZAR TABRIZ CO LTD,54926,54926-CONTROL AFZAR TABRIZ CO LTD
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731,NaN,NaN,NaN,<NA>,-
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,MECATRON MACHINERY CO LTD,54943,54943-MECATRON MACHINERY CO LTD


In [14]:
# pip install pandas openpyxl

In [15]:
# OFAC_data_check = pd.read_excel(r'C:\Users\wasia\Desktop\OFAC data listing\ofac_data_check.xlsx')
# OFAC_data_check.Alias = OFAC_data_check.Alias.fillna("")
# OFAC_data_check

In [16]:

# Importing the json ofac data
import requests

url = "https://data.trade.gov/downloadable_consolidated_screening_list/v1/consolidated.json"  # clean URL without query tags
resp = requests.get(url)
resp.raise_for_status()

data = resp.json()

# Again, adjust 'entries' to match the JSON structure you're working with
df_json = pd.json_normalize(data,"results")

df_json.head()


,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,gross_tonnage,gross_registered_tonnage,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities
0,"[{'address': '172 Xibin Rd, Ranghulu District,...","[KARAMAY URBAN CREDIT COOPERATIVES, KARAMAY CI...",15268,15268,"[{'type': 'SWIFT/BIC', 'number': 'CKLBCNBJ'}]",BANK OF KUNLUN CO LTD,[561-Related],United States financial institutions are prohi...,Capta List (CAP) - Treasury Department,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'address': '31 Jinrong Street, Xicheng Distr...","[CHINA TELECOM, CHINA TELECOM CO., LTD, CHINA ...",30882,30882,"[{'type': 'Unified Social Credit Code (USCC)',...",China Telecom Corporation Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[{'address': '5 Landianchang South Road Haidia...,"[Aero Engine Corp of China, AECC, CHINA AVIATI...",30902,30902,"[{'type': 'Unified Social Credit Code (USCC)',...",Aero Engine Corporation of China,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'address': '60th Floor, The Center, 99 Queen...",[CHINA MOBILE LTD],30923,30923,"[{'type': 'Equity Ticker', 'number': 'CHL US'}...",China Mobile Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'address': '75th Floor, The Center, 99 Queen...",[China Unicom Limited],30924,30924,"[{'type': 'Equity Ticker', 'number': 'CHU US'}...",China Unicom (Hong Kong) Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# with open(r'C:\UID Listing\OFAC.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
# df_all_data = pd.json_normalize(data['results'])
# df_all_data.head()



In [18]:
df_json.shape

(23300, 30)

In [19]:
# combining id and name column
df_json["ID-NAME"] = df_json["id"] + "-" + df_json["name"]
df_json.head()

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,gross_registered_tonnage,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME
0,"[{'address': '172 Xibin Rd, Ranghulu District,...","[KARAMAY URBAN CREDIT COOPERATIVES, KARAMAY CI...",15268,15268,"[{'type': 'SWIFT/BIC', 'number': 'CKLBCNBJ'}]",BANK OF KUNLUN CO LTD,[561-Related],United States financial institutions are prohi...,Capta List (CAP) - Treasury Department,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15268-BANK OF KUNLUN CO LTD
1,"[{'address': '31 Jinrong Street, Xicheng Distr...","[CHINA TELECOM, CHINA TELECOM CO., LTD, CHINA ...",30882,30882,"[{'type': 'Unified Social Credit Code (USCC)',...",China Telecom Corporation Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30882-China Telecom Corporation Limited
2,[{'address': '5 Landianchang South Road Haidia...,"[Aero Engine Corp of China, AECC, CHINA AVIATI...",30902,30902,"[{'type': 'Unified Social Credit Code (USCC)',...",Aero Engine Corporation of China,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30902-Aero Engine Corporation of China
3,"[{'address': '60th Floor, The Center, 99 Queen...",[CHINA MOBILE LTD],30923,30923,"[{'type': 'Equity Ticker', 'number': 'CHL US'}...",China Mobile Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30923-China Mobile Limited
4,"[{'address': '75th Floor, The Center, 99 Queen...",[China Unicom Limited],30924,30924,"[{'type': 'Equity Ticker', 'number': 'CHU US'}...",China Unicom (Hong Kong) Limited,[CMIC-EO13959],None,Non-SDN Chinese Military-Industrial Complex Co...,https://ofac.treasury.gov/other-ofac-sanctions...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30924-China Unicom (Hong Kong) Limited


In [20]:
df_json.shape

(23300, 31)

In [21]:
# df_json.to_excel('df_json.xlsx')

In [49]:

# filtering checking df_check in df_json
filtered_df = df_json[df_json['ID-NAME'].isin(df_checkv2['id-name'])]



# Create a mapping dictionary from df_check
name_to_source = dict(zip(df_checkv2['id-name'], df_checkv2['source']))
idname_to_alias = dict(zip(df_checkv2['id-name'], df_checkv2['alias']))


# Add the 'Source' column to filtered_df2 using map
filtered_df['source'] = filtered_df['ID-NAME'].map(name_to_source)
filtered_df['alias2'] = filtered_df['ID-NAME'].map(idname_to_alias)
filtered_df

C:\Users\wasia\AppData\Local\Temp\ipykernel_6036\1946133589.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['source'] = filtered_df['ID-NAME'].map(name_to_source)
C:\Users\wasia\AppData\Local\Temp\ipykernel_6036\1946133589.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['alias2'] = filtered_df['ID-NAME'].map(idname_to_alias)


,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22729,"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[],54926,54926,"[{'type': 'National ID No.', 'number': '102001...",CONTROL AFZAR TABRIZ CO LTD,"[NPWMD, IFSR]",(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54926-CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز
22731,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,..."
22739,"[{'city': 'Taichung City', 'country': 'TW', 'a...",[],54943,54943,"[{'type': 'Business Number', 'number': '242498...",MECATRON MACHINERY CO LTD,"[IFSR, NPWMD]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54943-MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司
22741,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...


In [45]:
filtered_df

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22729,"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[],54926,54926,"[{'type': 'National ID No.', 'number': '102001...",CONTROL AFZAR TABRIZ CO LTD,"[NPWMD, IFSR]",(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54926-CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز
22731,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,..."
22739,"[{'city': 'Taichung City', 'country': 'TW', 'a...",[],54943,54943,"[{'type': 'Business Number', 'number': '242498...",MECATRON MACHINERY CO LTD,"[IFSR, NPWMD]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54943-MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司
22741,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...


In [50]:
# droping duplicate cases based on same id-names
filtered_df2 = filtered_df.drop_duplicates(subset=['ID-NAME'], keep='last')
filtered_df2

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME,alias2
22729,"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[],54926,54926,"[{'type': 'National ID No.', 'number': '102001...",CONTROL AFZAR TABRIZ CO LTD,"[NPWMD, IFSR]",(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54926-CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز
22731,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADE HOUSHYAR, Javad, ALIZADEH-HOSHIAR, Ja...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,..."
22739,"[{'city': 'Taichung City', 'country': 'TW', 'a...",[],54943,54943,"[{'type': 'Business Number', 'number': '242498...",MECATRON MACHINERY CO LTD,"[IFSR, NPWMD]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54943-MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司
22741,"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[CHANGZHOU JOMARS INDUSTRIAL AUTOMATION CO LTD...,54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...


In [25]:
filtered_df2.columns

Index(['addresses', 'alt_names', 'entity_number', 'id', 'ids', 'name',
       'programs', 'remarks', 'source', 'source_information_url',
       'source_list_url', 'type', 'federal_register_notice', 'start_date',
       'end_date', 'standard_order', 'license_policy', 'license_requirement',
       'call_sign', 'vessel_type', 'gross_tonnage', 'gross_registered_tonnage',
       'vessel_flag', 'vessel_owner', 'title', 'citizenships',
       'dates_of_birth', 'country', 'places_of_birth', 'nationalities',
       'ID-NAME', 'alias2'],
      dtype='object')

In [51]:

# Convert alias2 strings into lists by splitting on ';', handle NaN safely
filtered_df2['alias2'] = filtered_df['alias2'].fillna('').apply(lambda x: [item.strip() for item in x.split(';') if item.strip()])

# Combine and remove duplicates using set, convert back to list for alias2 and alt_names columns
filtered_df2['alt_names'] = filtered_df2.apply(lambda row: list(set(row['alt_names']) | set(row['alias2'])), axis=1)


# Drop alias2 column not needed now
filtered_df2.drop(columns=['alias2'], inplace=True)

In [52]:
filtered_df2

,addresses,alt_names,entity_number,id,ids,name,programs,remarks,source,source_information_url,...,gross_registered_tonnage,vessel_flag,vessel_owner,title,citizenships,dates_of_birth,country,places_of_birth,nationalities,ID-NAME
22729,"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[Arabic: شرکت کنترل افزار تبريز],54926,54926,"[{'type': 'National ID No.', 'number': '102001...",CONTROL AFZAR TABRIZ CO LTD,"[NPWMD, IFSR]",(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,None,[],[],NaN,[],[],54926-CONTROL AFZAR TABRIZ CO LTD
22731,"[{'country': 'IR', 'address': None, 'city': No...","[ALIZADEH HOSHYAR, Javad Saeid, ALIZADE HOUSHY...",54930,54930,"[{'type': 'National ID No.', 'number': '137656...","ALIZADEH HOSHYAR, Javad","[NPWMD, IFSR]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,None,[],[1976-08-24],NaN,[Iran],[IR],"54930-ALIZADEH HOSHYAR, Javad"
22739,"[{'city': 'Taichung City', 'country': 'TW', 'a...",[Chinese Traditional: 龍馬精機股份有限公司],54943,54943,"[{'type': 'Business Number', 'number': '242498...",MECATRON MACHINERY CO LTD,"[IFSR, NPWMD]",(Linked To: CONTROL AFZAR TABRIZ CO LTD),https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,None,[],[],NaN,[],[],54943-MECATRON MACHINERY CO LTD
22741,"[{'city': 'Changzhou', 'country': 'CN', 'addre...","[Chinese Simplified: 常州乔懋工业自动化有限公司, CHANGZHOU ...",54945,54945,"[{'type': 'Unified Social Credit Code (USCC)',...",CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[IFSR, NPWMD]",(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,https://ofac.treasury.gov/recent-actions/20250731,https://home.treasury.gov/policy-issues/financ...,...,None,None,None,None,[],[],NaN,[],[],54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...


In [53]:
temp_df = filtered_df2[['ID-NAME','entity_number', 'type','name','alt_names',
                        'places_of_birth','addresses','source', 'programs',                
                      'remarks','vessel_type','vessel_flag','vessel_owner',
                      'citizenships','dates_of_birth','country','nationalities','ids']]
temp_df['remarks'] = temp_df['remarks'].fillna('').str.split(';')
temp_df = temp_df.reset_index(drop=True)
temp_df.head()

C:\Users\wasia\AppData\Local\Temp\ipykernel_6036\2754430856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['remarks'] = temp_df['remarks'].fillna('').str.split(';')


,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,vessel_type,vessel_flag,vessel_owner,citizenships,dates_of_birth,country,nationalities,ids
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,CONTROL AFZAR TABRIZ CO LTD,[Arabic: شرکت کنترل افزار تبريز],[],"[{'city': 'Tabriz', 'country': 'IR', 'address'...",https://ofac.treasury.gov/recent-actions/20250731,"[NPWMD, IFSR]",[(Linked To: IRAN AIRCRAFT MANUFACTURING INDUS...,None,None,None,[],[],NaN,[],"[{'type': 'National ID No.', 'number': '102001..."
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","[ALIZADEH HOSHYAR, Javad Saeid, ALIZADE HOUSHY...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",https://ofac.treasury.gov/recent-actions/20250731,"[NPWMD, IFSR]",[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],None,None,None,[],[1976-08-24],NaN,[IR],"[{'type': 'National ID No.', 'number': '137656..."
2,54943-MECATRON MACHINERY CO LTD,54943,Entity,MECATRON MACHINERY CO LTD,[Chinese Traditional: 龍馬精機股份有限公司],[],"[{'city': 'Taichung City', 'country': 'TW', 'a...",https://ofac.treasury.gov/recent-actions/20250731,"[IFSR, NPWMD]",[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],None,None,None,[],[],NaN,[],"[{'type': 'Business Number', 'number': '242498..."
3,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,"[Chinese Simplified: 常州乔懋工业自动化有限公司, CHANGZHOU ...",[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",https://ofac.treasury.gov/recent-actions/20250731,"[IFSR, NPWMD]",[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,None,None,None,[],[],NaN,[],"[{'type': 'Unified Social Credit Code (USCC)',..."


In [ ]:

temp_df.columns


Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'ids'],
      dtype='object')

In [ ]:
type(temp_df.ids[0])

list

In [ ]:
type(temp_df['remarks'].iloc[0])

list

In [72]:



# extracting sanction date from source link
temp_df['sanction_date'] = temp_df['source'].str.rsplit('/', n=1).str[-1].str[:8]

temp_df

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,sanction_date
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,[CONTROL AFZAR TABRIZ CO LTD],[Arabic: شرکت کنترل افزار تبريز],[],"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[https://ofac.treasury.gov/recent-actions/2025...,[NPWMD; IFSR],[(Linked To: IRAN AIRCRAFT MANUFACTURING INDUS...,...,19,USA,United States,Sanction,False,True,False,False,False,NaN
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"[ALIZADEH HOSHYAR, Javad]","[ALIZADEH HOSHYAR, Javad Saeid, ALIZADE HOUSHY...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",[https://ofac.treasury.gov/recent-actions/2025...,[NPWMD; IFSR],[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],...,19,USA,United States,Sanction,False,True,False,False,False,NaN
2,54943-MECATRON MACHINERY CO LTD,54943,Entity,[MECATRON MACHINERY CO LTD],[Chinese Traditional: 龍馬精機股份有限公司],[],"[{'city': 'Taichung City', 'country': 'TW', 'a...",[https://ofac.treasury.gov/recent-actions/2025...,[IFSR; NPWMD],[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],...,19,USA,United States,Sanction,False,True,False,False,False,NaN
3,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,[CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD],"[Chinese Simplified: 常州乔懋工业自动化有限公司, CHANGZHOU ...",[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[https://ofac.treasury.gov/recent-actions/2025...,[IFSR; NPWMD],[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,...,19,USA,United States,Sanction,False,True,False,False,False,NaN


In [54]:
# extracting order_id from program name if available
import re
# function to extract from program name if found
def extract_order_ids(program_list):
    pattern = r'EO\d+'
    matches = []
    for item in program_list:
        if item:
            matches.extend(re.findall(pattern, item))
    # Replace 'EO' with 'E.O.' in all matched strings
    matches = [m.replace('EO', 'E.O.') for m in matches]
    return "; ".join(matches)

temp_df['order_id'] = temp_df['programs'].apply(extract_order_ids)

In [55]:
# joining all program separated by ;
temp_df['programs'] = temp_df['programs'].apply(
    lambda x: '; '.join(str(i) for i in x if i is not None) if isinstance(x, list) else x
)

In [56]:
temp_df['programs']

0    NPWMD; IFSR
1    NPWMD; IFSR
2    IFSR; NPWMD
3    IFSR; NPWMD
Name: programs, dtype: object

In [ ]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'ids', 'sanction_date',
       'order_id'],
      dtype='object')

In [57]:
#convert string list to actual list
import ast

def parse_ids_column(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return []
    return val

temp_df['ids'] = temp_df['ids'].apply(parse_ids_column)

In [58]:
# this function extract all usefull values from ids into seperate column and extract all E.O. form ids key and type key
import pandas as pd
import re

wanted_types = [
    'Phone Number',
    'Target Type',
    'Website',
    'Organization Established Date',
    'Organization Type:',
    'Email Address',
    'Gender'
]

eo_patterns = [
    'Executive Order 14024',
    'Executive Order 13662',
    'Executive Order 13846'
]

remove_types = [
    'Additional Sanctions Information -',
    'Effective Date (CMIC)',
    'Effective Date (EO 14024 Directive 1a):',
    'Effective Date (EO 14024 Directive 3):',
    'Effective Date (EO 14024 Directive 2):',
    'Secondary sanctions risk:',
    'Transactions Prohibited For Persons Owned or Controlled By U.S. Financial Institutions:',
    'Issuer Name'
] + wanted_types  # Also remove extracted wanted types from ids

# Prepare a normalized list for comparison (lowercase, stripped, no trailing colon)
remove_types_lower = [t.lower().strip().rstrip(':') for t in remove_types]

def process_ids(row):
    ids_list = row.get('ids')
    if not isinstance(ids_list, list):
        # If ids_list is not a list, return None/NaN for all extracted fields
        data = {f"{k}_extracted": None for k in wanted_types}
        data['order_id_extracted'] = row.get('order_id', None)
        data['ids_extracted'] = ids_list
        return pd.Series(data)

    extracted_values = {f"{k}_extracted": None for k in wanted_types}
    order_ids = []

    for d in ids_list:
        t = d.get('type')
        n = d.get('number')

        # Extract wanted types values
        if t and t in wanted_types:
            extracted_values[f"{t}_extracted"] = n

        # Extract EO codes from type field
        if t and any(pattern in t for pattern in eo_patterns):
            match = re.search(r'Executive Order (\d+)', t)
            if match:
                eo_code = f"E.O.{match.group(1)}"
                order_ids.append(eo_code)

        # NEW: Extract EO codes from number field (string)
        if n and isinstance(n, str):
            matches = re.findall(r'Executive Order (\d+)', n)
            for m in matches:
                eo_code = f"E.O.{m}"
                order_ids.append(eo_code)

    # Combine existing order_id with extracted EO codes, remove duplicates
    existing_order_id = row.get('order_id')
    all_order_ids = [existing_order_id] if existing_order_id else []
    all_order_ids.extend(order_ids)
    # Remove duplicates and join with "; "
    combined_order_id = "; ".join(sorted(set(filter(None, all_order_ids)))) if all_order_ids else None

    # Filter out unwanted 'type's robustly (ignore case, strip spaces and trailing colon)
    filtered_ids = [
        d for d in ids_list
        if d.get('type') is None or d.get('type').lower().strip().rstrip(':') not in remove_types_lower
    ]

    extracted_values['order_id_extracted'] = combined_order_id
    extracted_values['ids_extracted'] = filtered_ids
    return pd.Series(extracted_values)

# Usage example:

temp_df = temp_df.join(temp_df.apply(process_ids, axis=1))


In [65]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [66]:
temp_df = temp_df.drop(columns=['ids','order_id'])

KeyError: "['ids', 'order_id'] not found in axis"

In [67]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [68]:
# add fixed columns  
temp_df['source_description'] = 'United States Department of the Treasury'
temp_df['sanction_authority'] = 'U.S. Department of Treasury - Office of Foreign Assets Control'
temp_df['sanction_list_name'] = 'Specially Designated Nationals (SDN) - Treasury Department'
temp_df['sanction_authority_ids'] = '19'
temp_df['sanction_authority_country'] = 'USA'
temp_df['subject_country'] = 'United States'
temp_df['source_type'] = 'Sanction'
temp_df['is_pep'] = 'False'
temp_df['is_sanction'] = 'True'
temp_df['is_watchlist'] = 'False'
temp_df['is_enforcement'] = 'False'
temp_df['is_apc'] = 'False'


In [69]:
temp_df

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_list_name,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,[CONTROL AFZAR TABRIZ CO LTD],[Arabic: شرکت کنترل افزار تبريز],[],"[{'city': 'Tabriz', 'country': 'IR', 'address'...",[https://ofac.treasury.gov/recent-actions/2025...,[NPWMD; IFSR],[(Linked To: IRAN AIRCRAFT MANUFACTURING INDUS...,...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
1,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"[ALIZADEH HOSHYAR, Javad]","[ALIZADEH HOSHYAR, Javad Saeid, ALIZADE HOUSHY...",[Iran],"[{'country': 'IR', 'address': None, 'city': No...",[https://ofac.treasury.gov/recent-actions/2025...,[NPWMD; IFSR],[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
2,54943-MECATRON MACHINERY CO LTD,54943,Entity,[MECATRON MACHINERY CO LTD],[Chinese Traditional: 龍馬精機股份有限公司],[],"[{'city': 'Taichung City', 'country': 'TW', 'a...",[https://ofac.treasury.gov/recent-actions/2025...,[IFSR; NPWMD],[(Linked To: CONTROL AFZAR TABRIZ CO LTD)],...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
3,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,[CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD],"[Chinese Simplified: 常州乔懋工业自动化有限公司, CHANGZHOU ...",[],"[{'city': 'Changzhou', 'country': 'CN', 'addre...",[https://ofac.treasury.gov/recent-actions/2025...,[IFSR; NPWMD],[(Linked To: JOEMARS MACHINERY AND ELECTRIC IN...,...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False


In [70]:
temp_df.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'],
      dtype='object')

In [73]:
# Ensure all columns have lists and replace NaN or other non-list types with empty lists

columns_to_process = ['name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'sanction_date'
       , 'Phone Number_extracted', 'Target Type_extracted',
       'Website_extracted', 'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc'
                      ]

for col in columns_to_process:
    temp_df[col] = temp_df[col].apply(
        lambda x: x if isinstance(x, list) else [str(x)] if pd.notnull(x) else []
    )

# Step 1: Get the max length of lists across the columns
max_len = temp_df[columns_to_process].applymap(len).max(axis=1)

# Step 2: Pad the shorter lists with None
for col in columns_to_process:
    temp_df[col] = temp_df.apply(lambda row: row[col] + [None] * (max_len[row.name] - len(row[col])), axis=1)

# Step 3: Explode the lists into separate rows
temp_df_expanded = temp_df.explode(columns_to_process, ignore_index=True)

# Display the expanded DataFrame
temp_df_expanded


C:\Users\wasia\AppData\Local\Temp\ipykernel_6036\3165593618.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_len = temp_df[columns_to_process].applymap(len).max(axis=1)


,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,sanction_date
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,...,19,USA,United States,Sanction,False,True,False,False,False,None
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tehran', 'country': 'IR', 'address':...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,19,USA,United States,Sanction,False,True,False,False,False,None
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,54943-MECATRON MACHINERY CO LTD,54943,Entity,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,"{'city': 'Taichung City', 'country': 'TW', 'ad...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,19,USA,United States,Sanction,False,True,False,False,False,None
9,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司,None,"{'city': 'Changzhou', 'country': 'CN', 'addres...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,...,19,USA,United States,Sanction,False,True,False,False,False,None


In [ ]:

temp_df_expanded

,ID-NAME,entity_number,type,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,sanction_list_name,sanction_authority_ids,sanction_authority_country,subject_country,source_type,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tehran', 'country': 'IR', 'address':...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,54942-CLIFTON TRADING LIMITED,54942,Entity,CLIFTON TRADING LIMITED,None,None,"{'city': 'Hong Kong', 'country': 'CN', 'addres...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Specially Designated Nationals (SDN) - Treasur...,19,USA,United States,Sanction,False,True,False,False,False
9,54942-CLIFTON TRADING LIMITED,54942,Entity,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [74]:
temp_df_expanded.columns

Index(['ID-NAME', 'entity_number', 'type', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_date'],
      dtype='object')

In [75]:
# Renaming country and type column to solve the naming confilict with country and type in address dictionary expanding into new columns  

temp_df_expanded.rename(columns={'country': 'country1', 'type': 'type1'}, inplace=True)
temp_df_expanded.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_date'],
      dtype='object')

In [ ]:
temp_df_expanded.addresses.loc[0]

{'city': 'Tabriz',
 'country': 'IR',
 'address': 'No. 66, The End of Asia Street 4, Foreign Investment Town, Tabriz Road, Sufian',
 'state': None,
 'postal_code': None}

In [76]:

# Handle None values and apply pd.Series to expand the dictionary into new seperate columns here "addresses" and "ids" are dictionary columns

temp_df_expanded1 = temp_df_expanded.join(temp_df_expanded['addresses'].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({})))


In [77]:
# Renaming the country name which we get from expanding address dictionary to solve the naming confilict with country in ids dictionary 
temp_df_expanded1 = temp_df_expanded1.rename(columns={'country': 'address_country'})

In [ ]:
temp_df_expanded1.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'sanction_date',
       'Phone Number_extracted', 'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'city', 'address_country',
       'address', 'state', 'postal_code'],
      dtype='object')

In [78]:
# expanding ids column dictionary into separate columns 
temp_df_expanded1 = temp_df_expanded1.join(temp_df_expanded['ids_extracted'].apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series({})))
temp_df_expanded1

,ID-NAME,entity_number,type1,name,alt_names,places_of_birth,addresses,source,programs,remarks,...,city,address_country,address,state,postal_code,type,number,country,issue_date,expiration_date
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,...,Tabriz,IR,"No. 66, The End of Asia Street 4, Foreign Inve...",None,None,National ID No.,10200172180,IR,NaN,NaN
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tehran', 'country': 'IR', 'address':...",None,None,None,...,Tehran,IR,"West Unit, Second Floor, No. 247, Beginning of...",None,None,Registration Number,11374,IR,NaN,NaN
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Entity,None,None,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",None,None,None,...,Tabriz,IR,"Unit 2, 3rd Floor, Valiasr Commercial Complex,...",None,None,Chamber of Commerce Number,18103377,IR,NaN,NaN
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,None,IR,None,None,None,National ID No.,1376561999,IR,NaN,NaN
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54943-MECATRON MACHINERY CO LTD,54943,Entity,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,"{'city': 'Taichung City', 'country': 'TW', 'ad...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Taichung City,TW,"No. 26-4, Sanxian Rd, Lianhe Village",Taichung Province,421017,Business Number,24249847,TW,NaN,NaN
9,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Entity,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司,None,"{'city': 'Changzhou', 'country': 'CN', 'addres...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,...,Changzhou,CN,"No. 678, Xin Chang Road, Xin Zha Township",Jiangsu,None,Unified Social Credit Code (USCC),913204006081307514,CN,NaN,NaN


In [79]:
temp_df_expanded1.columns


Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_date', 'city',
       'address_country', 'address', 'state', 'postal_code', 'type', 'number',
       'country', 'issue_date', 'expiration_date'],
      dtype='object')

In [80]:
temp_df_expanded1.address_country

0      IR
1      IR
2      IR
3      IR
4     NaN
5     NaN
6     NaN
7     NaN
8      TW
9      CN
10    NaN
11    NaN
12    NaN
Name: address_country, dtype: object

In [81]:
temp_df_expanded1.columns

Index(['ID-NAME', 'entity_number', 'type1', 'name', 'alt_names',
       'places_of_birth', 'addresses', 'source', 'programs', 'remarks',
       'vessel_type', 'vessel_flag', 'vessel_owner', 'citizenships',
       'dates_of_birth', 'country1', 'nationalities', 'Phone Number_extracted',
       'Target Type_extracted', 'Website_extracted',
       'Organization Established Date_extracted',
       'Organization Type:_extracted', 'Email Address_extracted',
       'Gender_extracted', 'order_id_extracted', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_date', 'city',
       'address_country', 'address', 'state', 'postal_code', 'type', 'number',
       'country', 'issue_date', 'expiration_date'],
      dtype='object')

In [82]:
temp_df_expanded2 = temp_df_expanded1.rename(columns={'country': 'identifier_country',
                                  'address': 'address_street',
                                  'city':'address_city',
                                  'state':'address_province',
                                 'postal_code':'address_postal_code',
                                  'type':'identifier_name',
                                 'number':'identifier_value',
                                 'issue_date':'identifier_issue_date_date',
                                 'expiration_date':'identifier_expiry_date_date',
                                 'citizenships':'citizenship_country',
                                 'nationalities':'nationality_country',
                                 'alt_names':'alias_name',
                                 'type1':'type',
                                 'programs':'sanction_programme_name',
                                 'remarks':'position',
                                 'vessel_type':'vessel_type',
                                 'vessel_flag':'current_country_flag',
                                 'sanction_date':'sanction_action_date_date',
                                 'Phone Number_extracted':'contact_number',
                                 'Website_extracted':'website',
                                 'Organization Established Date_extracted':'date_of_incorporation_date',
                                 'Organization Type:_extracted':'sanction_remarks',
                                 'Email Address_extracted':'email_id',
                                 'Gender_extracted':'gender',
                                 'order_id_extracted':'sanction_order_number',
                                 'entity_number':'ID',
                                
                                 })

temp_df_expanded2['type'] = temp_df_expanded2['type'].replace('Entity','Organization')

temp_df_expanded2

,ID-NAME,ID,type,name,alias_name,places_of_birth,addresses,source,sanction_programme_name,position,...,address_city,address_country,address_street,address_province,address_postal_code,identifier_name,identifier_value,identifier_country,identifier_issue_date_date,identifier_expiry_date_date
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: IRAN AIRCRAFT MANUFACTURING INDUST...,...,Tabriz,IR,"No. 66, The End of Asia Street 4, Foreign Inve...",None,None,National ID No.,10200172180,IR,NaN,NaN
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,"{'city': 'Tehran', 'country': 'IR', 'address':...",None,None,None,...,Tehran,IR,"West Unit, Second Floor, No. 247, Beginning of...",None,None,Registration Number,11374,IR,NaN,NaN
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,"{'city': 'Tabriz', 'country': 'IR', 'address':...",None,None,None,...,Tabriz,IR,"Unit 2, 3rd Floor, Valiasr Commercial Complex,...",None,None,Chamber of Commerce Number,18103377,IR,NaN,NaN
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Iran,"{'country': 'IR', 'address': None, 'city': Non...",https://ofac.treasury.gov/recent-actions/20250731,NPWMD; IFSR,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,None,IR,None,None,None,National ID No.,1376561999,IR,NaN,NaN
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,"{'city': 'Taichung City', 'country': 'TW', 'ad...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: CONTROL AFZAR TABRIZ CO LTD),...,Taichung City,TW,"No. 26-4, Sanxian Rd, Lianhe Village",Taichung Province,421017,Business Number,24249847,TW,NaN,NaN
9,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Organization,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司,None,"{'city': 'Changzhou', 'country': 'CN', 'addres...",https://ofac.treasury.gov/recent-actions/20250731,IFSR; NPWMD,(Linked To: JOEMARS MACHINERY AND ELECTRIC IND...,...,Changzhou,CN,"No. 678, Xin Chang Road, Xin Zha Township",Jiangsu,None,Unified Social Credit Code (USCC),913204006081307514,CN,NaN,NaN


In [83]:
# spiliting birth address into different columns
import numpy as np
def split_pob_with_type(value):
    if pd.isna(value):
        return pd.Series([pd.NA, pd.NA, pd.NA])
    address_type = 'Birth'
    if ',' in value:
        parts = value.rsplit(',', 1)
        return pd.Series([address_type, parts[0].strip(), parts[1].strip()])
    else:
        return pd.Series([address_type, pd.NA, value.strip()])

temp_df_expanded2[['pob_address_type', 'pob_address', 'pob_country']] = temp_df_expanded2['places_of_birth'].apply(split_pob_with_type)
temp_df_expanded2['address_type']= np.nan

In [84]:
temp_df_expanded2.pob_country

0     <NA>
1     <NA>
2     <NA>
3     Iran
4     <NA>
5     <NA>
6     <NA>
7     <NA>
8     <NA>
9     <NA>
10    <NA>
11    <NA>
12    <NA>
Name: pob_country, dtype: object

In [85]:
temp_df_expanded2.columns


Index(['ID-NAME', 'ID', 'type', 'name', 'alias_name', 'places_of_birth',
       'addresses', 'source', 'sanction_programme_name', 'position',
       'vessel_type', 'current_country_flag', 'vessel_owner',
       'citizenship_country', 'dates_of_birth', 'country1',
       'nationality_country', 'contact_number', 'Target Type_extracted',
       'website', 'date_of_incorporation_date', 'sanction_remarks', 'email_id',
       'gender', 'sanction_order_number', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_action_date_date',
       'address_city', 'address_country', 'address_street', 'address_province',
       'address_postal_code', 'identifier_name', 'identifier_value',
       'identifier_country', 'identifier_issue_date_date',
       'identifier_expiry_date_dat

In [86]:
# importing countrycode df
country_code_df = pd.read_excel(r"Country_code.xlsx",
                   keep_default_na=False, na_values=[])
country_code_df


,record,Country1,Country,countrycode3,countrycode2
0,,Afghanistan,Afghanistan,AFG,AF
1,,Aland Islands,Aland Islands,ALA,AX
2,,Albania,Albania,ALB,AL
3,,Algeria,Algeria,DZA,DZ
4,,American Samoa,American Samoa,ASM,AS
...,...,...,...,...,...
234,,Wallis and Futuna,Wallis and Futuna,WLF,WF
235,,Western Sahara,Western Sahara,ESH,EH
236,,Yemen,Yemen,YEM,YE
237,,Zambia,Zambia,ZMB,ZM


In [87]:

# Create a mapping dictionary from country_code_df
code_to_name = dict(zip(country_code_df['countrycode2'], country_code_df['Country']))

# Replace country codes with country names in the specified columns of temp_df_expanded2
columns_to_map = ['address_country', 'nationality_country', 'citizenship_country','identifier_country']
for col in columns_to_map:
    temp_df_expanded2[col] = temp_df_expanded2[col].map(code_to_name).fillna(temp_df_expanded2[col])

# Create mapping dictionaries for countrycode3
code_to_countrycode3 = dict(zip(country_code_df['Country'], country_code_df['countrycode3']))

# Add new columns for 'countrycode3' based on 'country', 'nationality','pob_country' and 'citizen'
columns_to_map2 = ['address_country', 'nationality_country', 'citizenship_country','identifier_country','pob_country']
for col in columns_to_map2:
    temp_df_expanded2[f'{col}_code'] = temp_df_expanded2[col].map(code_to_countrycode3).fillna(temp_df_expanded2[col])

    
    
temp_df_expanded2.pob_country

0     <NA>
1     <NA>
2     <NA>
3     Iran
4     <NA>
5     <NA>
6     <NA>
7     <NA>
8     <NA>
9     <NA>
10    <NA>
11    <NA>
12    <NA>
Name: pob_country, dtype: object

In [88]:
temp_df_expanded2.columns

Index(['ID-NAME', 'ID', 'type', 'name', 'alias_name', 'places_of_birth',
       'addresses', 'source', 'sanction_programme_name', 'position',
       'vessel_type', 'current_country_flag', 'vessel_owner',
       'citizenship_country', 'dates_of_birth', 'country1',
       'nationality_country', 'contact_number', 'Target Type_extracted',
       'website', 'date_of_incorporation_date', 'sanction_remarks', 'email_id',
       'gender', 'sanction_order_number', 'ids_extracted',
       'source_description', 'sanction_authority', 'sanction_list_name',
       'sanction_authority_ids', 'sanction_authority_country',
       'subject_country', 'source_type', 'is_pep', 'is_sanction',
       'is_watchlist', 'is_enforcement', 'is_apc', 'sanction_action_date_date',
       'address_city', 'address_country', 'address_street', 'address_province',
       'address_postal_code', 'identifier_name', 'identifier_value',
       'identifier_country', 'identifier_issue_date_date',
       'identifier_expiry_date_dat

In [89]:
# spliting the sanction date

temp_df_expanded2['sanction_action_date_month'] = temp_df_expanded2['sanction_action_date_date'].str[4:6]       # 5th and 6th characters
temp_df_expanded2['sanction_action_date_year'] = temp_df_expanded2['sanction_action_date_date'].str[:4]     # Left 4 characters
temp_df_expanded2['sanction_action_date_date'] = temp_df_expanded2['sanction_action_date_date'].str[6:8]       # 7th and 8th characters

In [90]:
# selecting important columns except pob address and rearranging them in order
temp_df_expanded3 = temp_df_expanded2[['ID-NAME', 'ID', 'type', 'name', 'alias_name',
                                       'gender','dates_of_birth','citizenship_country','citizenship_country_code',
                                       'nationality_country','nationality_country_code','date_of_incorporation_date',
                                       'vessel_type','current_country_flag','address_type',
                                       'address_street','address_city', 'address_province', 'address_postal_code',
                                       'address_country','address_country_code','contact_number','email_id',
                                       'website','identifier_name','identifier_value','identifier_country',
                                       'identifier_country_code','identifier_issue_date_date','identifier_expiry_date_date',
                                       'position','sanction_authority','sanction_authority_ids','sanction_authority_country',
                                       'sanction_list_name','sanction_action_date_date','sanction_action_date_month',
                                       'sanction_action_date_year','sanction_order_number',
                                       'sanction_programme_name','source_type','source','source_description',
                                       'is_pep', 'is_sanction', 'is_watchlist', 'is_enforcement', 'is_apc',
                                       'subject_country'
                                        ]]


temp_df_expanded3

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,None,None,None,None,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,None,None,None,None,...,IFSR; NPWMD,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
9,54945-CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION ...,54945,Organization,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司,None,None,None,None,None,...,IFSR; NPWMD,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States


In [91]:
# selecting seperate pob address columns and not nan values
pob_df = temp_df_expanded2[['ID-NAME','ID','type','pob_address_type', 'pob_address','pob_country', 'pob_country_code']]
pob_df = pob_df[pob_df['pob_country'].notna()]
pob_df

,ID-NAME,ID,type,pob_address_type,pob_address,pob_country,pob_country_code
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,Birth,<NA>,Iran,IRN


In [92]:
pob_df = pob_df.rename(columns={
    'pob_address_type': 'address_type',
    'pob_address': 'address_street',
    'pob_country': 'address_country',
    'pob_country_code': 'address_country_code'
})
pob_df

,ID-NAME,ID,type,address_type,address_street,address_country,address_country_code
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,Birth,<NA>,Iran,IRN


In [93]:
# adding pob_df value below temp_df_expanded3
temp_df_expanded4 = pd.concat([temp_df_expanded3, pob_df], ignore_index=True)
# sort based on ID column and 
temp_df_expanded4 = temp_df_expanded4.sort_values(by=['ID', 'name'],na_position='last').reset_index(drop=True)
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,None,None,None,None,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,None,None,None,None,...,IFSR; NPWMD,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States


In [94]:
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,None,None,None,None,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,None,None,None,None,...,IFSR; NPWMD,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States


In [95]:
# extracting the column whic need to be sorted from main temp_df_expanded4
sort_df = temp_df_expanded4[['ID-NAME','ID','type','address_type','address_street','address_city','address_province','address_postal_code','address_country','address_country_code']]
# Sort by entity_number ascending and address_country with NaNs last
df_sorted = sort_df.sort_values(
    by=['ID', 'address_country'],
    na_position='last'
).reset_index(drop=True)
df_sorted = df_sorted[['address_type','address_street','address_city','address_province','address_postal_code','address_country','address_country_code']]
df_sorted

,address_type,address_street,address_city,address_province,address_postal_code,address_country,address_country_code
0,NaN,"No. 66, The End of Asia Street 4, Foreign Inve...",Tabriz,None,None,Iran,IRN
1,NaN,"West Unit, Second Floor, No. 247, Beginning of...",Tehran,None,None,Iran,IRN
2,NaN,"Unit 2, 3rd Floor, Valiasr Commercial Complex,...",Tabriz,None,None,Iran,IRN
3,NaN,None,None,None,None,Iran,IRN
4,Birth,NaN,NaN,NaN,NaN,Iran,IRN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,"No. 26-4, Sanxian Rd, Lianhe Village",Taichung City,Taichung Province,421017,Taiwan,TWN


In [96]:
# removing unsorted address columns from temp_df_expanded4
temp_df_expanded4 = temp_df_expanded4.drop(df_sorted.columns, axis=1)
temp_df_expanded4

,ID-NAME,ID,type,name,alias_name,gender,dates_of_birth,citizenship_country,citizenship_country_code,nationality_country,...,sanction_programme_name,source_type,source,source_description,is_pep,is_sanction,is_watchlist,is_enforcement,is_apc,subject_country
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,None,None,None,None,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,NPWMD; IFSR,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,None,None,None,None,...,IFSR; NPWMD,Sanction,https://ofac.treasury.gov/recent-actions/20250731,United States Department of the Treasury,False,True,False,False,False,United States


In [97]:
# final step adding sorted address including pob into temp_df_expanded4 as we have sorted both temp_df_expanded4 and df_sorted so it will aligned
concatenated_df = pd.concat([temp_df_expanded4, df_sorted], axis=1)
concatenated_df = concatenated_df.rename(columns={'name': 'entity_name','dates_of_birth':'date_of_birth_year'})
concatenated_df

,ID-NAME,ID,type,entity_name,alias_name,gender,date_of_birth_year,citizenship_country,citizenship_country_code,nationality_country,...,is_enforcement,is_apc,subject_country,address_type,address_street,address_city,address_province,address_postal_code,address_country,address_country_code
0,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز,None,None,None,None,None,...,False,False,United States,NaN,"No. 66, The End of Asia Street 4, Foreign Inve...",Tabriz,None,None,Iran,IRN
1,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,NaN,"West Unit, Second Floor, No. 247, Beginning of...",Tehran,None,None,Iran,IRN
2,54926-CONTROL AFZAR TABRIZ CO LTD,54926,Organization,None,None,None,None,None,None,None,...,None,None,None,NaN,"Unit 2, 3rd Floor, Valiasr Commercial Complex,...",Tabriz,None,None,Iran,IRN
3,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,"ALIZADEH HOSHYAR, Javad","ALIZADEH HOSHYAR, Javad Saeid",Male,1976-08-24,None,None,Iran,...,False,False,United States,NaN,None,None,None,None,Iran,IRN
4,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADE HOUSHYAR, Javad",None,None,None,None,None,...,None,None,None,Birth,NaN,NaN,NaN,NaN,Iran,IRN
5,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH-HOSHIAR, Javad",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,"ALIZADEH HUSHYAR, Javad",None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,None,Arabic: جواد عليزاده هوشيار,None,None,None,None,None,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"54930-ALIZADEH HOSHYAR, Javad",54930,Individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54943-MECATRON MACHINERY CO LTD,54943,Organization,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司,None,None,None,None,None,...,False,False,United States,NaN,"No. 26-4, Sanxian Rd, Lianhe Village",Taichung City,Taichung Province,421017,Taiwan,TWN


In [98]:
# importing format of ofac listing
df_format = pd.read_excel('OFAC_format.xlsx')
df_format

,ID,uid,merged_uids,type,entity_name,first_name,middle_name,last_name,alias_type,alias_name,...,Errors Update,Allocated to,Status (Error/Updated/No Change),Remarks,Date.1,UK SECO UN EU,Iden,Address,Title,full name


In [100]:
# Filter concatenated_df based on 'type'
df_org = concatenated_df[concatenated_df['type'] == 'Organization']
df_ind = concatenated_df[concatenated_df['type'] == 'Individual']

# Append to df_format to align columns and add missing columns as NaN
df_org_final = pd.concat([df_format, df_org], ignore_index=True, sort=False)
df_ind_final = pd.concat([df_format, df_ind], ignore_index=True, sort=False)

# Export to Excel with two sheets
with pd.ExcelWriter('final_output.xlsx') as writer:
    df_org_final.to_excel(writer, sheet_name='organization', index=False)
    df_ind_final.to_excel(writer, sheet_name='individual', index=False)